# 🧪 **Swarm** Demo with Pre-Built Agents

In this demo, we’re going to explore how to build a simple multi-agent system using LangGraph’s Swarm library.
We'll create two AI agents with different roles who can hand off tasks to each other dynamically — without using any tools.
This is perfect for showcasing agent collaboration, decision-making, and handoff logic using just prompts and messages.

By the end of this notebook, you’ll understand how to:

* Create agents using `create_react_agent`

* Connect them in a swarm using create_swarm

* Enable dynamic handoffs between agents

* Run a conversation where agents decide who should respond next

## 📦 Step 1: Install the necessary libraries

We’re making sure LangChain, LangGraph, and the OpenAI wrapper are on the latest version so every API we call later is available. The -U flag upgrades anything that’s already installed. We will also need the laggraph-swarm library on its latest version.

In [1]:
!pip install -U \
    langchain==0.3.27 \
    langgraph==0.6.4 \
    langchain-openai==0.3.28 \
    langgraph-swarm==0.0.14 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

## 🧠 Step 2: Import libraries

In this step, we're bringing in all the tools we need to build our multi-agent system.

🧩 What we’re importing:

* `AzureChatOpenAI` – This is the actual language model that our agents will use to think, reason, and generate responses.

* `InMemorySaver` – A simple way to store conversation history in memory. In this case, we will use short-term memory so the system remembers past messages during the session.

* `create_react_agent` – This is a handy function that lets us quickly create an agent with tools and logic baked in.

* `create_handoff_tool` – This allows one agent to pass the conversation to another agent when it’s no longer the right one to handle the task.

* `create_swarm` – This function brings all the agents together into a Swarm, where they can work together and hand off tasks as needed. It’s what we’ll use to build the collaborative environment.

In [2]:
from langchain_openai import AzureChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm

## 🔐 Step 3: Set up your OpenAI API key

The agent will talk to OpenAI’s GPT-4o-mini model. Putting your key in an environment variable lets every downstream cell authenticate automatically without hard-coding secrets into the notebook.

In [ ]:
import os
os.environ['AZURE_OPENAI_API_KEY'] = '<your-key>'
os.environ['AZURE_OPENAI_ENDPOINT'] = '<your-endpoint>'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

## 🧠 Step 4: Load the Model

`AzureChatOpenAI` gives us a convenient Python object that wraps the GPT-4o-mini API. Setting `temperature=0.7` makes answers a little more creative while still fairly reliable.

In [5]:
model = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",
    model="gpt-4o-mini",
    temperature=0.7
)

## 👥 Step 5: Create Agent 1 – the task-focused agent
Agent 1 is focused on structured queries or task-specific help.
If it receives a message that feels more conversational, it will hand the task off to Agent 2.
We use create_handoff_tool() to make this handoff behavior possible.

In [6]:
# Agent 1: Handles task-related queries, can hand off to Agent 2
agent_1 = create_react_agent(
    model=model,
    tools=[
        create_handoff_tool(
            agent_name="Agent2",
            description="For more casual or open-ended conversation."
        ),
    ],
    prompt="You are Agent 1. You are focused on helping the user with structured or task-specific queries. If the message is conversational or outside your scope, hand it off to Agent 2.",
    name="Agent1",
)

## 💬 Step 6: Create Agent 2 – the casual conversation agent
Agent 2 handles open-ended and general conversation.
If the user starts asking for help with a task or something more structured, Agent 2 will hand it back to Agent 1.

In [7]:
# Agent 2: Handles open-ended or casual conversation, can hand off to Agent 1
agent_2 = create_react_agent(
    model=model,
    tools=[
        create_handoff_tool(
            agent_name="Agent1",
            description="For task-related or structured queries."
        ),
    ],
    prompt="You are Agent 2. You specialize in casual conversation and open-ended questions. If the user needs structured help, hand it off to Agent 1.",
    name="Agent2",
)

# 💾 Step 7: Set up memory checkpointing
LangGraph uses checkpoints to keep track of the conversation across turns.
We'll use an in-memory saver here.

In [8]:
# Use in-memory checkpointing
checkpointer = InMemorySaver()

## 🤖 Step 8: Create the Swarm
Now that we’ve defined both agents, we’ll bring them together into a Swarm.

We also specify which agent should start the conversation—in this case, we’re starting with Agent 2.

In [9]:
# Create the swarm
workflow = create_swarm(
    agents=[agent_1, agent_2],
    default_active_agent="Agent2",  # Starting with Agent 2
)

✅ At this point, we’ve defined the structure of our agent team—but it’s not running yet.

## 🛠️ Step 9: Compile the Swarm
Before we can use our swarm in a live conversation, we need to compile it.
Compiling finalizes the graph and prepares it for interaction.
We also plug in our checkpointer, so the system remembers what was said in this thread.

In [10]:
# Compile the app
app = workflow.compile(checkpointer=checkpointer)

## 🧵 Step 10: Set the thread ID for this conversation
Each conversation needs a thread_id to track the message history.
We'll keep it simple and use "1".

In [11]:
# Set up conversation config
config = {"configurable": {"thread_id": "1"}}

## 💬 Step 11: Start the conversation – a casual message
Let’s simulate a user message to start a casual chat.
Since it's not a task, Agent 2 should handle it.

In [12]:
# User opens the conversation
turn_1 = app.invoke(
    {"messages": [{"role": "user", "content": "Hey, I just wanted to chat a bit."}]},
    config,
)
print(turn_1)

{'messages': [HumanMessage(content='Hey, I just wanted to chat a bit.', additional_kwargs={}, response_metadata={}, id='f59f0a6a-2ada-4123-b759-9cac5a6ff555'), AIMessage(content="Of course! I'm here for a chat. What's on your mind?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 81, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-Cbob8ektcvi96Yb3KtDD20Ik8Tl27', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, name='Agent2', id='run--9e62e8e8-326b-448e-acd9-ebc7cf481b20-0', usage_metadata={'input_tokens': 81, 'output_tokens': 16, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'ou

In [13]:
first_response = turn_1["messages"][-1]
print("Agent:", first_response.content)

Agent: Of course! I'm here for a chat. What's on your mind?


## 🔄 Step 12: Follow-up – a task-based request
Now the user switches gears and asks for help with a task.
Agent 2 should recognize this and hand off the conversation to Agent 1.

In [14]:
# User asks a task-like question
turn_2 = app.invoke(
    {"messages": [{"role": "user", "content": "Actually, can you help me write a short email?"}]},
    config,
)
print(turn_2)

{'messages': [HumanMessage(content='Hey, I just wanted to chat a bit.', additional_kwargs={}, response_metadata={}, id='f59f0a6a-2ada-4123-b759-9cac5a6ff555'), AIMessage(content="Of course! I'm here for a chat. What's on your mind?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 81, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-Cbob8ektcvi96Yb3KtDD20Ik8Tl27', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, name='Agent2', id='run--9e62e8e8-326b-448e-acd9-ebc7cf481b20-0', usage_metadata={'input_tokens': 81, 'output_tokens': 16, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'ou

In [15]:
second_response = turn_2["messages"][-1]
print("Agent:", second_response.content)

Agent: Sure! What would you like the email to say, and who is it intended for?
